# Import

In [1]:
import torch

# IoU

In [2]:
from yolo.src.metrics import iou

In [3]:
pred = torch.tensor([
    [3, 3, 7, 7],
    [1, -1, 3, 2],
    [1, 1, 2, 2]
])

true = torch.tensor([
    [4, 4, 8, 8],
    [2, -1, 3, 2],
    [3, 3, 4, 4]
])

In [4]:
iou(pred, true, format='pascal_voc')

tensor([0.3913, 0.5000, 0.0000])